# C. Elegans DNA

Import libraries for later use

In [ ]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

Read the C.Elegens .csv file. We add our own headers - labels stands for whether there is a splice site or not and the DNA is a string representing the DNA

In [ ]:
df = pd.read_csv('exercise_data/C_elegans_acc_seq.csv', header=None, names=['labels', 'DNA'])

### Doing the Test-Train-Split

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(28)
train, test = train_test_split(df, test_size=0.2)

# Make a copy of the raw dna sequences for later use with Shogun
train_raw = np.array(train)
test_raw = np.array(test)
X_train = train_raw[:,1]
y_train = train_raw[:,0]
X_test = test_raw[:,1]
y_test = test_raw[:,0]

In [ ]:
# Check the label proportions are similar. stratify=True for splitting threw a weird exception
print(100*np.sum(df['labels']==1)/df.shape[0])
print(100*np.sum(train['labels']==1)/train.shape[0])
print(100*np.sum(test['labels']==1)/test.shape[0])

### Mapping DNA to a vector

We will map the DNA into a vector, by mapping each Character (A,T,C,G) into a one-hot vector and then concatenating all these vectors together. As we have a string of 82 Characters this gives us a final vector of length 328

In [ ]:
import utility
train['DNA'] = train['DNA'].map(utility.map_dna_into_vector)
test['DNA'] = test['DNA'].map(utility.map_dna_into_vector)

### Creating DataFrame for later Evaluation

In [ ]:
f1_eval_df = pd.DataFrame(data=[], columns=['Name', 'AUROC_cv', 'f1_cv', 'AUROC_test', 'AUPRC_test', 'f1_test'])
auroc_eval_df = pd.DataFrame(data=[], columns=['Name', 'AUROC_cv', 'f1_cv', 'AUROC_test', 'AUPRC_test', 'f1_test'])

## Models

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def evaluate_model(model, params, df, train, test, metric, half=0, svc=0):
    # Put Data into a readable Matrix format
    train_data = np.vstack(train['DNA'].values)
    test_data  = np.vstack(test['DNA'].values)
    
    if half:
        if half == 1:
            train_data = train_data[:, :int(train_data.shape[1]/2) - 4]
            test_data = test_data[:, :int(test_data.shape[1]/2) - 4]
        elif half == 2:
            train_data = train_data[:, (int(train_data.shape[1]/2) + 8):]
            test_data = test_data[:, (int(test_data.shape[1]/2) + 8):]
        else:
            raise ValueError('half must take values 0|1|2')
    
    # Create Instance of our Model
    m = model()
    
    # Specify metrics
    scoring = {'roc_auc': 'roc_auc', 'f1': 'f1'}
    
    # Search for the best params in our model and print the best score
    clf = GridSearchCV(m, params, scoring=scoring, refit=metric, cv=5, n_jobs=-1)
    clf.fit(train_data, train['labels'].values)
    print(f"The best score was: {clf.best_score_}")
    
    # Extract cv metric results
    results = clf.cv_results_
    cv_dict = {'AUROC_cv':results['mean_test_roc_auc'][clf.best_index_],
               'f1_cv':results['mean_test_f1'][clf.best_index_]}
    
    # Train our best model on the whole train-dataset
    best_estimator = model(**clf.best_params_)
    best_estimator.fit(train_data, train['labels'].values)
    
    # Evaluate on the Test set
    pred_val = best_estimator.predict(test_data)
    true_val = test['labels'].values
    if svc:
        pred_scores = best_estimator.decision_function(test_data)
    else:
        pred_scores = best_estimator.predict_proba(test_data)[:,1]
        
    auroc_test, auprc_test, f1_test = utility.get_scores(true_val, pred_val, pred_scores)
    test_dict = {'AUROC_test': auroc_test, 
                 'AUPRC_test': auprc_test, 
                 'f1_test': f1_test}
    
    # Append to our Dataframe
    new_row = {'Name': model.__name__ + '_half' + str(half)} if half else {'Name': model.__name__}
    new_row.update(cv_dict)
    new_row.update(test_dict)
    df = df.append(new_row, ignore_index=True)
    return (best_estimator, df)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [1, 10, 100],
    'class_weight': [None, 'balanced']
}

In [ ]:
lg_best_estimator, f1_eval_df = evaluate_model(LogisticRegression, params, f1_eval_df, train, test, 'f1')
_, auroc_eval_df = evaluate_model(LogisticRegression, params, auroc_eval_df, train, test, 'roc_auc')

### SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
params = {'kernel': ['linear', 'rbf', 'poly'],
          'C': [0.1, 1, 10, 100],
          'class_weight': [None, 'balanced'],
          'gamma': ['auto', 'scale']
         }

In [ ]:
svc_best_estimator, f1_eval_df = evaluate_model(SVC, params, f1_eval_df, train, test, 'f1', svc=1)
_, auroc_eval_df = evaluate_model(SVC, params, auroc_eval_df, train, test, 'roc_auc', svc=1)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
params = {'n_estimators':[10, 50, 100, 500],
          'class_weight': [None, 'balanced', 'balanced_subsample']
}

In [ ]:
rfc_best_estimator, f1_eval_df = evaluate_model(RandomForestClassifier, params, f1_eval_df, train, test, 'f1')
_, auroc_eval_df = evaluate_model(RandomForestClassifier, params, auroc_eval_df, train, test, 'roc_auc')

###  Gaussian Process Classifer

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, PairwiseKernel

In [ ]:
params = {
    'kernel' : [RBF(), PairwiseKernel()]
}

In [ ]:
gpc_best_estimator, f1_eval_df = evaluate_model(GaussianProcessClassifier, params, f1_eval_df, train, test, 'f1')
_, auroc_eval_df = evaluate_model(GaussianProcessClassifier, params, auroc_eval_df, train, test, 'roc_auc')

## SVM + String kernels (using SHOGUN)

In [ ]:
import shogun as sg

features_train = sg.StringCharFeatures(list(X_train), sg.DNA)
labels_train = sg.BinaryLabels(y_train.astype(int))
features_test = sg.StringCharFeatures(list(X_test), sg.DNA)
labels_test = sg.BinaryLabels(y_test.astype(int))

### Weighted Degree Kernel

We perform optimization via grid search

In [ ]:
#Root
#param_tree_root = sg.ModelSelectionParameters()

#Parameter C
#C = sg.ModelSelectionParameters("C")
#param_tree_root.append_child(C)
#C.build_values(1, 10, sg.R_LINEAR, 1, 2)
#C.set_values(1,2,3)

#kernel = sg.WeightedDegreeStringKernel(features_train, features_train, kernel_degree)
#svm = sg.LibSVM(C, kernel, labels_train)

#C.print_tree()

In [ ]:
C = 2
kernel_degree = 3

kernel = sg.WeightedDegreeStringKernel(features_train, features_train, kernel_degree)
svm = sg.LibSVM(C, kernel, labels_train)

# Cross validation
stratified_split = sg.StratifiedCrossValidationSplitting(labels_train, 5)
metric = sg.F1Measure()
cross = sg.CrossValidation(svm, features_train, labels_train, stratified_split, metric)
# 25 runs and 95% confidence intervals
cross.set_num_runs(25)
cross.set_autolock(False)
result = cross.evaluate()
cv_score = sg.CrossValidationResult.obtain_from_generic(result).get_mean()
print("CV score", metric.get_name(), cv_score)

Train on whole train dataset to evaluate test performance

In [ ]:
svm.train()
pred = svm.apply(features_test)
pred_val = pred.get_labels()
pred_scores = pred.get_values()
auroc, auprc, f1_test = utility.get_scores(y_test.astype(int), pred_val, pred_scores)
    
# Append to our Dataframe
f1_eval_df = f1_eval_df.append({'Name': 'WDK_' + str(kernel_degree), 'f1_cv':cv_score, 'AUROC_test':auroc, 'AUPRC_test': auprc, 'f1_test':f1_test}, ignore_index=True)

### Fixed Degree String Kernel

In [ ]:
C = 2
kernel_degree = 3

kernel = sg.FixedDegreeStringKernel(features_train, features_train, kernel_degree)
svm = sg.LibSVM(C, kernel, labels_train)

# Cross validation
stratified_split = sg.StratifiedCrossValidationSplitting(labels_train, 5)
metric = sg.F1Measure()
cross = sg.CrossValidation(svm, features_train, labels_train, stratified_split, metric)
# 25 runs and 95% confidence intervals
cross.set_num_runs(25)
cross.set_autolock(False)
result = cross.evaluate()
cv_score = sg.CrossValidationResult.obtain_from_generic(result).get_mean()
print("CV score", metric.get_name(), cv_score)

Train on whole train dataset to evaluate test performance

In [ ]:
svm.train()
pred = svm.apply(features_test)
pred_val = pred.get_labels()
pred_scores = pred.get_values()
auroc, auprc, f1_test = utility.get_scores(y_test.astype(int), pred_val, pred_scores)
    
# Append to our Dataframe
f1_eval_df = f1_eval_df.append({'Name': 'FDK_' + str(kernel_degree), 'f1_cv':cv_score, 'AUROC_test':auroc, 'AUPRC_test': auprc, 'f1_test':f1_test}, ignore_index=True)

### Oligo String Kernel

In [ ]:
C = 2
kernel_degree = 3
kernel_width = 10

kernel = sg.OligoStringKernel(features_train, features_train, kernel_degree, kernel_width)
svm = sg.LibSVM(C, kernel, labels_train)

# Cross validation
stratified_split = sg.StratifiedCrossValidationSplitting(labels_train, 5)
metric = sg.F1Measure()
cross = sg.CrossValidation(svm, features_train, labels_train, stratified_split, metric)
# 25 runs and 95% confidence intervals
cross.set_num_runs(1)
cross.set_autolock(False)
result = cross.evaluate()
cv_score = sg.CrossValidationResult.obtain_from_generic(result).get_mean()
print("CV score", metric.get_name(), cv_score)

Train on whole train dataset to evaluate test performance

In [ ]:
svm.train()
pred = svm.apply(features_test)
pred_val = pred.get_labels()
pred_scores = pred.get_values()
auroc, auprc, f1_test = utility.get_scores(y_test.astype(int), pred_val, pred_scores)
    
# Append to our Dataframe
f1_eval_df = f1_eval_df.append({'Name': 'OSK_' + str(kernel_degree) + '_' + str(kernel_width),
                                'f1_cv':cv_score, 'AUROC_test':auroc, 'AUPRC_test': auprc, 'f1_test':f1_test}, ignore_index=True)

### Weighted Degree Position String Kernel

In [ ]:
C = 2
kernel_degree = 1

kernel = sg.WeightedDegreePositionStringKernel(features_train, features_train, kernel_degree)
svm = sg.LibSVM(C, kernel, labels_train)

# Cross validation
stratified_split = sg.StratifiedCrossValidationSplitting(labels_train, 5)
metric = sg.F1Measure()
cross = sg.CrossValidation(svm, features_train, labels_train, stratified_split, metric)
# 25 runs and 95% confidence intervals
cross.set_num_runs(25)
cross.set_autolock(False)
result = cross.evaluate()
cv_score = sg.CrossValidationResult.obtain_from_generic(result).get_mean()
print("CV score", metric.get_name(), cv_score)

Train on whole train dataset to evaluate test performance

In [ ]:
svm.train()
pred = svm.apply(features_test)
pred_val = pred.get_labels()
pred_scores = pred.get_values()
auroc, auprc, f1_test = utility.get_scores(y_test.astype(int), pred_val, pred_scores)
    
# Append to our Dataframe
f1_eval_df = f1_eval_df.append({'Name': 'WDPSK_' + str(kernel_degree), 'f1_cv':cv_score, 'AUROC_test':auroc, 'AUPRC_test': auprc, 'f1_test':f1_test}, ignore_index=True)

### Multiple Kernel Learning

Here we combine the better performing previous kernels mixed via a weighted average

In [ ]:
if False:
    C = 1

    #poly_kernel = sg.PolyKernel(dot_features, dot_features, 3, True, 10)
    WD_kernel = sg.WeightedDegreeStringKernel(3)
    #WDP_kernel = sg.WeightedDegreePositionStringKernel()

    combined_kernel = sg.CombinedKernel()
    combined_kernel.append_kernel(WD_kernel)
    #combined_kernel.append_kernel(WDP_kernel)
    combined_kernel.init(features_train, features_train)

    svm = sg.LibSVM(C, combined_kernel, labels_train)

    # Cross validation
    stratified_split = sg.StratifiedCrossValidationSplitting(labels_train, 5)
    metric = sg.F1Measure()
    cross = sg.CrossValidation(svm, features_train, labels_train, stratified_split, metric)
    # 25 runs and 95% confidence intervals
    cross.set_num_runs(25)
    cross.set_autolock(False)
    result = cross.evaluate()
    cv_score = sg.CrossValidationResult.obtain_from_generic(result).get_mean()
    print("CV score", metric.get_name(), cv_score)

Train on whole train dataset to evaluate test performance

In [ ]:
if False:
    svm.train()
    pred = svm.apply(features_test)
    pred_val = pred.get_labels()
    pred_scores = pred.get_values()
    auroc, auprc, f1_test = utility.get_scores(y_test.astype(int), pred_val, pred_scores)

    # Append to our Dataframe
    f1_eval_df = f1_eval_df.append({'Name': 'MKL', 'f1_cv':cv_score, 'AUROC_test':auroc, 'AUPRC_test': auprc, 'f1_test':f1_test}, ignore_index=True)

### DL Model

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import BatchNormalization,Conv1D,Input,Add,Dense,Flatten
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam

def add_RB(x):
    xout=BatchNormalization()(x)
    xout=Conv1D(filters=32,kernel_size=11,dilation_rate=1,padding='same',activation='relu')(x)
    xout=BatchNormalization()(xout)
    xout=Conv1D(filters=32,kernel_size=11,dilation_rate=1,padding='same',activation='relu')(xout)
    return xout

In [ ]:
tf.reset_default_graph()
x=Input(shape=[328,1])

x1=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(x)

xrb=add_RB(x1)

x2=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(xrb)
x3=Conv1D(filters=32,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(x1)

xout=Conv1D(filters=1,kernel_size=1,dilation_rate=1,padding='same',activation='relu')(Add()([x2,x3]))
xout=Flatten()(xout)
xout=Dense(units=1,activation='sigmoid')(xout)

model=Model(x,xout)
model.compile(optimizer=Adam(),loss='binary_crossentropy')
class_wt={0:1,1:15}

train_data = np.vstack(train['DNA'].values)[:,:,None]
test_data  = np.vstack(test['DNA'].values)[:,:,None]

train_val=train['labels'].values
train_val[train_val==-1]=0
model.fit(x=train_data,y=train_val,batch_size=64,epochs=20,class_weight=class_wt)

In [ ]:
pred_scores=model.predict(test_data)
pred_val=(pred_scores>0.5).astype(np.int)
true_val=test['labels']
true_val[true_val==-1]=0
dl_mtr=utility.get_scores(true_val,pred_val,pred_scores)

In [ ]:
f1_eval_df = f1_eval_df.append({'Name': 'DL Model', 'AUROC_test':dl_mtr[0], 'AUPRC_test': dl_mtr[1], 'f1_test':dl_mtr[2]}, ignore_index=True)

### SVC half string

In this section we only use the first or second half of the dna sequences in order to assess the influence of each substring in the accuracy of predictions.

First substring:

In [ ]:
params = {'kernel': ['linear', 'rbf', 'poly'],
          'C': [0.1, 1, 10, 100],
          'class_weight': [None, 'balanced'],
          'gamma': ['auto', 'scale']
         }

In [ ]:
svc_best_estimator_half, f1_eval_df = evaluate_model(SVC, params, f1_eval_df, train, test, 'f1', half=1, svc=1)
_, auroc_eval_df = evaluate_model(SVC, params, auroc_eval_df, train, test, 'roc_auc', half=1, svc=1)

Second substring:

In [ ]:
params = {'kernel': ['linear', 'rbf', 'poly'],
          'C': [0.1, 1, 10, 100],
          'class_weight': [None, 'balanced'],
          'gamma': ['auto', 'scale']
         }

In [ ]:
svc_best_estimator_half, f1_eval_df = evaluate_model(SVC, params, f1_eval_df, train, test, 'f1', half=2, svc=1)
_, auroc_eval_df = evaluate_model(SVC, params, auroc_eval_df, train, test, 'roc_auc', half=2, svc=1)

## Evaluation

In [ ]:
f1_eval_df

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

best_estimator = svc_best_estimator
predicted_values = best_estimator.predict(np.vstack(test['DNA'].values))
predicted_scores = best_estimator.decision_function(np.vstack(test['DNA'].values))
true_values = test['labels']

# compute ROC curve
fpr, tpr, thresholds_roc = roc_curve(true_values, predicted_scores)
roc_auc = auc(fpr, tpr)
precision, recall, thresholds_prc = precision_recall_curve(true_values, predicted_scores)

# compute precision-recall curve
auprc = auc(recall, precision)
precision_random, recall_random, thresholds_random = precision_recall_curve(true_values, np.random.rand(len(true_values)))
auprc_random = auc(recall_random, precision_random)
other_scores_validation = [roc_auc, auprc, auprc_random]

# plot curves
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity / False Positive Rate')
plt.ylabel('Sensitivity / True Positive Rate')
plt.title('Receiver Operating Characteristic curve')
plt.legend(loc="lower right")
#plt.savefig('./models/' + experiment_id + '/' + model_name + '_roc_curve_validation.png')

plt.figure()
lw = 2
plt.plot(recall, precision, color='darkorange',
                     lw=lw, label='AUPRC curve (area = %0.2f)' % auprc)
plt.plot(recall_random, precision_random, color='navy', linestyle='--',
                     lw=lw, label='random AUPRC curve (area = %0.2f)' % auprc_random)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower right")



## Conclusion

- Kernelized SVM and DL works the best, but not much better than other simpler models such as logistic regression.
- String kernels didn't improve over polynomial kernel.
- First half of the dna sequences holds most of the relevant information for prediction